In [4]:
from qiskit import *
from qiskit.circuit.library import StatePreparation
from qiskit.circuit.library import UnitaryGate
from qiskit.circuit.library import HamiltonianGate
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
simulator = AerSimulator()

import numpy as np
from numpy import linalg
import math

from sympy import Matrix
from sympy.physics.quantum import TensorProduct

import matplotlib.pyplot as plt

## The Problem

The Ellipsoid Method(EM) solves convex optimization problems. It attemts to find a solution to the problem $\textbf{min}(c^T x) \textit{ s.t. } x \in P = \{x \in R^n: Ax \geq b\}$ where $A$ and $b$ have integer inputs. If $P$ is empty, then EM will say so. Otherwise, it will find a feasible point, and then optimize from there.

### Initialize

In order to run ellipsoid, we need a few objects. The first is the maximum number of iterations it is going to run and the second is the initial ellipsoid.

By finding the smallest area of which $P$ can exist within, called $v$, and the largest area for a which a point of $P$ must exist within, called $V$, we can calculate the maximum number of iterations. We can then use a sphere of volume $V$ centered at $0$ as our first ellipsoid.

In [331]:
def Initialize(A,b):
    U = float(max(A.max(),b.max()))
    n = float(len(A[0]))
    
    small_v = n**(-n) * (n*U)**((-n**(2))*(n+1))
    
    big_V = (2*n)**n *(n*U)**(n**2)
    max_time = math.ceil(2*(n+1)*math.log(big_V/small_v))
    
    radius = big_V/math.pi
    D_0 = radius*np.identity(int(n))
    
    return(max_time, D_0)

### Conditioning and Rescaling

In order to ensure EM works properly, we must condition and scale $A$ and $b$. First, we calculate a perturbation $e$ which affects $b$ which generates a new polyhedra $P_e$. This polyhedra will be empty if $P$ is empty, and fully dimensional if $P$ is not. Then, we add 0s to the rows of $A$ so we are working in the space $\mathbb{R}^{2^n}$, this is so it can be represented as qubits. Finally, we normalize the rows of $A$ so that they can be encoded as quantum states. We scale $b$ accordingly 


In [314]:
def Rescale(A,b):
    A_rescale = []
    b_rescale = []
    
    # Perturbs $b$ by a small amount so it is fully dimensional.
    n = len(A[0])
    U = float(max(A.max(),b.max()))
    
    perturbation =  1/(2*(n+1) * ((n+1)*U)**(n+1))
    
    # Takes the rows of A and adds extra 0 variables so we are in R^(2^n) for some n
    small_length = len(A[0])
    large_length = 2**(math.ceil(math.log(small_length,2)))  # The smallest power of 2 which is >= small_length
    difference = large_length - small_length
    
    # Takes the rows of A as vectors and normalizes them
    for row in range(len(A)):
        length = 0.0
        for entry in range(len(A[row])):
            length = length+(float(A[row][entry])**2)
        length = math.sqrt(length)
        
        temp_A = list(A[row]/length)
        i = 0
        while i < difference:
            temp_A.append(0.0)
            i = i+1
        
        temp_b = (b[row]-perturbation)/length  # This is where Perturbation is used
        
        A_rescale.append(temp_A)
        b_rescale.append(temp_b)
    
    new_A = np.array(A_rescale)
    new_b = np.array(b_rescale)
    
    return(new_A, new_b)

### Swap Test

Given two quantum states $|\psi \rangle$ and $|\phi \rangle$, the swap test can calculate $\langle \psi | \phi \rangle$ to within $\epsilon$ with $\mathcal{O}(\frac{1}{\epsilon^2})$ shots. Adapted from "Quantum Algorithms to Matrix Multiplication" by C. Shao

In [315]:
def InnerProduct(a_row,x_vector,b_target,error):
    # Rescale x_vector so it can be encoded as a quantum state. Adjust b_target to maintain
    x_norm = np.linalg.norm(x_vector)
    x_normalized = x_vector/x_norm
    b_normalized = b_target/x_norm
    
    # Prepair states a_row and x_normalized
    plus_state = np.array([[math.sqrt(2)/2], [math.sqrt(2)/2]])
    minus_state = np.array([[math.sqrt(2)/2], [-math.sqrt(2)/2]])
    
    a_state = np.tensordot(plus_state,a_row,axes=0)
    a_state = list(a_state)
    a_state = list(a_state[0][0]) + list(a_state[1][0])
    a_state = (math.sqrt(2)/2) * np.array(a_state)
    
    x_state = np.tensordot(minus_state,x_normalized,axes=0)
    x_state = list(x_state)
    x_state = list(x_state[0][0]) + list(x_state[1][0])
    x_state = (math.sqrt(2)/2) * np.array(x_state)
    
    q_state = a_state + x_state
    
    # Create Cirucit
    length = int(math.log(len(q_state), 2))
    qc = QuantumCircuit(length,1)
    qc.initialize(q_state)
    
    qc.measure(length-1,0)
    
    # Perform measurements
    job = transpile(qc,backend=simulator)
    shots = int(math.ceil(1/((0.9*error)**2)))
    result = simulator.run(job, shots=shots).result()
    
    # Data Processing
    inner_product = 2*(result.data()['counts']['0x0']/shots) - 1
    difference = abs(inner_product - b_normalized)
    
    if difference >= error:
        return a_row
    else:
        return "Groovy"

In [316]:
InnerProduct(M[0],M[1],-0.8,0.01)

array([0.8, 0.6, 0. , 0. , 0. , 0. , 0. , 0. ])

### Violated $a_i$

This function uses the SWAP test to determine if $Ax \geq b$. If $Ax \geq b$, then it passes. Otherwise, if returns the violated row of $A$.

In [317]:
# Given A_matrix, x_vector, and b_vector, will tell which row of A_matrix has been violated, if any
def ViolatedRow(A_matrix,x_vector,b_vector,error):
    for rows in range(len(A)):
        if type(InnerProduct(A[rows],x,b[rows],e)) == str:
            pass
        else:
            return A[rows]
        
    return 'Groovy'

### Hamiltonian Gate

Given a matrix $D$, this function will return the matrix $\begin{pmatirx} 0 & D \\ D & 0\end{pmatrix}$. This Hamiltonian matrix can then be used as quantum gate.

In [318]:
def HamiltonianMatrix(D):
    zero = np.zeros((len(D),len(D)))
    right = np.concatenate((D, zero))
    left = np.concatenate((zero, np.transpose(D)))
    
    Hamiltonian = np.hstack((left,right))
    
    return Hamiltonian

### The Ellipsoid Method

In [319]:
# Given matrix A and vector b, give vector x such that Ax >= b with probability alpha
def EllipsoidFeasibility(A,b,alpha):
    max_time, D_0 = Initialize(A,b)
    
    return D_0

In [326]:
# Test Data
A = np.array([[4.0, 3.0, 0.0, 0.0, 0.0],
              [1.0, 0.0, 0.0, 0.0, 0.0],
              [1.0, 1.0, 1.0, 1.0, 0.0]])

x = np.array([3.0, 2.0, 1.0, 0.0])

b = np.array([18.0, 3.0, 6.0])

e = .01

f = 3.0

D = np.array([[f,0,0,1],
              [0,f,0,0],
              [0,0,f,0],
              [0,0,0,f]])

M,c = Rescale(A,b)

In [332]:
Initialize(A,b)

OverflowError: cannot convert float infinity to integer